In [ ]:
def meta(dump_file):   
    import lief
    from construct import (Struct, Int32ul, Int64ul, Bytes, this, Padding, 
                        Array, If, Computed, Pointer, Enum, Hex, Int16ul, Int8ul)

    # 定义MINIDUMP头结构
    MINIDUMP_HEADER = Struct(
        "signature" / Hex(Int32ul),  # 文件签名，应为'MDMP'(0x504d444d)
        "version" / Int32ul,        # 版本号(通常为42899)
        "stream_count" / Int32ul,   # 数据流(stream)的数量
        "stream_directory_rva" / Int32ul,  # 流目录表的文件偏移
        "checksum" / Int32ul,       # 校验和(通常为0)
        "timestamp" / Int32ul,      # 转储文件创建时间(Unix时间戳)
        "flags" / Hex(Int64ul)      # 标志位，表示包含哪些信息
    )

    # 定义流目录项结构
    STREAM_DIRECTORY = Struct(
        "stream_type" / Enum(Int32ul,
            UnusedStream=0,
            ReservedStream0=1,
            ReservedStream1=2,
            ThreadListStream=3,      # 线程列表
            ModuleListStream=4,      # 加载模块列表
            MemoryListStream=5,      # 内存区域列表
            ExceptionStream=6,       # 异常信息
            SystemInfoStream=7,      # 系统信息
            ThreadExListStream=8,
            Memory64ListStream=9,
            CommentStreamA=10,
            CommentStreamW=11,
            HandleDataStream=12,     # 句柄信息
            FunctionTableStream=13,
            UnloadedModuleListStream=14,
            MiscInfoStream=15,       # 杂项信息
            MemoryInfoListStream=16, # 内存区域详细信息
            ThreadInfoListStream=17,
            HandleOperationListStream=18,
            LastReservedStream=0xffff
        ),
        "data_size" / Int32ul,      # 数据流大小(字节)
        "rva" / Int32ul             # 数据流在文件中的偏移
    )

    # 定义系统信息结构
    SYSTEM_INFO = Struct(
        "processor_architecture" / Enum(Int16ul,
            PROCESSOR_ARCHITECTURE_INTEL=0,
            PROCESSOR_ARCHITECTURE_ARM=5,
            PROCESSOR_ARCHITECTURE_IA64=6,
            PROCESSOR_ARCHITECTURE_AMD64=9,
            PROCESSOR_ARCHITECTURE_ARM64=12,
            PROCESSOR_ARCHITECTURE_UNKNOWN=0xffff
        ),
        "processor_level" / Int16ul,    # CPU家族(如6=奔腾Pro/II/III)
        "processor_revision" / Int16ul, # CPU型号和步进
        Padding(2),                     # 对齐填充
        "number_of_processors" / Int8ul,# 逻辑处理器数量
        "product_type" / Enum(Int8ul,   # 系统类型
            VER_NT_WORKSTATION=1,
            VER_NT_DOMAIN_CONTROLLER=2,
            VER_NT_SERVER=3
        ),
        "major_version" / Int32ul,      # 主版本号(如10 for Win10)
        "minor_version" / Int32ul,      # 次版本号
        "build_number" / Int32ul,       # 构建号
        "platform_id" / Enum(Int32ul,  # 平台ID
            VER_PLATFORM_WIN32s=0,
            VER_PLATFORM_WIN32_WINDOWS=1,
            VER_PLATFORM_WIN32_NT=2
        ),
        "csd_version_rva" / Int32ul,    # 服务包信息字符串偏移
        "suite_mask" / Int16ul,         # 系统套件标志
        "reserved2" / Int16ul
    )

    # 定义模块信息结构
    MODULE = Struct(
        "base_of_image" / Int64ul,      # 模块基地址
        "size_of_image" / Int32ul,      # 模块大小
        "checksum" / Int32ul,           # PE校验和
        "timestamp" / Int32ul,          # PE时间戳
        "module_name_rva" / Int32ul,    # 模块名偏移
        "version_info" / Struct(        # 版本信息
            "signature" / Int32ul,
            "struct_version" / Int32ul,
            "file_version_ms" / Int32ul, # 文件版本(高32位)
            "file_version_ls" / Int32ul, # 文件版本(低32位)
            "product_version_ms" / Int32ul,
            "product_version_ls" / Int32ul,
            "file_flags_mask" / Int32ul,
            "file_flags" / Int32ul,
            "file_os" / Int32ul,
            "file_type" / Int32ul,
            "file_subtype" / Int32ul,
            "file_date_ms" / Int32ul,
            "file_date_ls" / Int32ul
        ),
        "cv_record" / Struct(           # CodeView调试信息
            "data_size" / Int32ul,
            "rva" / Int32ul
        ),
        "misc_record" / Struct(         # 杂项调试信息
            "data_size" / Int32ul,
            "rva" / Int32ul
        ),
        "reserved0" / Int64ul,
        "reserved1" / Int64ul
    )

    # 定义线程信息结构
    THREAD = Struct(
        "thread_id" / Int32ul,          # 线程ID
        "suspend_count" / Int32ul,      # 挂起计数
        "priority_class" / Int32ul,     # 优先级类
        "priority" / Int32ul,           # 线程优先级
        "teb" / Int64ul,                # 线程环境块地址
        "stack" / Struct(               # 栈信息
            "start_of_memory_range" / Int64ul,
            "memory" / Struct(
                "data_size" / Int32ul,
                "rva" / Int32ul
            )
        ),
        "thread_context" / Struct(      # 线程上下文
            "data_size" / Int32ul,
            "rva" / Int32ul
        )
    )

    # 定义异常信息结构
    EXCEPTION = Struct(
        "exception_code" / Hex(Int32ul), # 异常代码
        "exception_flags" / Int32ul,
        "exception_record" / Int64ul,   # 指向关联异常记录的指针
        "exception_address" / Int64ul,  # 异常发生地址
        "number_parameters" / Int32ul,  # 异常参数数量
        "unused_alignment" / Int32ul,
        "exception_information" / Array(15, Int64ul) # 异常参数数组
    )

    # 定义内存区域信息结构
    MEMORY_DESCRIPTOR = Struct(
        "start_of_memory_range" / Int64ul, # 内存起始地址
        "memory" / Struct(              # 内存数据位置
            "data_size" / Int32ul,      # 数据大小
            "rva" / Int32ul             # 文件偏移
        )
    )

    def parse_minidump(dump_file):
        """解析minidump文件并提取所有元数据信息"""
        with open(dump_file, "rb") as f:
            # 1. 解析头部信息
            header = MINIDUMP_HEADER.parse_stream(f)
            
            # 验证文件签名
            if header.signature != 0x504d444d:  # 'MDMP'
                raise ValueError("无效的minidump文件签名")
            
            # 2. 读取流目录表
            f.seek(header.stream_directory_rva)
            streams = []
            for _ in range(header.stream_count):
                stream = STREAM_DIRECTORY.parse_stream(f)
                streams.append(stream)
            
            # 3. 解析各个数据流
            metadata = {
                "header": header,
                "system_info": None,
                "modules": [],
                "threads": [],
                "exception": None,
                "memory_regions": []
            }
            
            for stream in streams:
                f.seek(stream.rva)
                
                if stream.stream_type == "SystemInfoStream":
                    # 解析系统信息
                    metadata["system_info"] = SYSTEM_INFO.parse_stream(f)
                    
                elif stream.stream_type == "ModuleListStream":
                    # 解析模块列表
                    module_count = Int32ul.parse_stream(f)
                    for _ in range(module_count):
                        module = MODULE.parse_stream(f)
                        
                        # 读取模块名称
                        pos = f.tell()
                        f.seek(module.module_name_rva)
                        module_name_length = Int32ul.parse_stream(f)
                        module_name = Bytes(module_name_length).parse_stream(f).decode('utf-16-le').rstrip('\x00')
                        f.seek(pos)
                        
                        metadata["modules"].append({
                            "name": module_name,
                            "base_address": hex(module.base_of_image),
                            "size": module.size_of_image,
                            "timestamp": module.timestamp,
                            "version": f"{module.version_info.file_version_ms >> 16}.{module.version_info.file_version_ms & 0xFFFF}."
                                    f"{module.version_info.file_version_ls >> 16}.{module.version_info.file_version_ls & 0xFFFF}"
                        })
                        
                elif stream.stream_type == "ThreadListStream":
                    # 解析线程列表
                    thread_count = Int32ul.parse_stream(f)
                    for _ in range(thread_count):
                        thread = THREAD.parse_stream(f)
                        metadata["threads"].append({
                            "id": thread.thread_id,
                            "suspend_count": thread.suspend_count,
                            "priority": thread.priority,
                            "teb": hex(thread.teb),
                            "stack_size": thread.stack.memory.data_size
                        })
                        
                elif stream.stream_type == "ExceptionStream":
                    # 解析异常信息
                    thread_id = Int32ul.parse_stream(f)
                    exception = EXCEPTION.parse_stream(f)
                    metadata["exception"] = {
                        "thread_id": thread_id,
                        "code": exception.exception_code,
                        "address": hex(exception.exception_address),
                        "parameters": [hex(p) for p in exception.exception_information[:exception.number_parameters]]
                    }
                    
                elif stream.stream_type == "MemoryListStream":
                    # 解析内存区域列表
                    region_count = Int32ul.parse_stream(f)
                    for _ in range(region_count):
                        region = MEMORY_DESCRIPTOR.parse_stream(f)
                        metadata["memory_regions"].append({
                            "start_address": hex(region.start_of_memory_range),
                            "size": region.memory.data_size,
                            "file_offset": region.memory.rva
                        })
            
            return metadata


        
    metadata = parse_minidump(dump_file)
    
    # 打印摘要信息
    print(f"Minidump分析报告: {dump_file}")
    print(f"创建时间: {metadata['header'].timestamp} (Unix时间戳)")
    print("\n系统信息:")
    print(f"  CPU架构: {metadata['system_info'].processor_architecture}")
    print(f"  Windows版本: {metadata['system_info'].major_version}.{metadata['system_info'].minor_version}")
    print(f"  构建号: {metadata['system_info'].build_number}")
    
    print("\n加载模块:")
    for module in metadata['modules'][:5]:  # 只显示前5个模块
        print(f"  {module['name']} @ {module['base_address']} (大小: {module['size']} bytes)")
    
    print("\n线程信息:")
    for thread in metadata['threads'][:3]:  # 只显示前3个线程
        print(f"  线程ID: {thread['id']}, 栈大小: {thread['stack_size']} bytes")
        
    if metadata['exception']:
        print("\n异常信息:")
        print(f"  异常代码: 0x{metadata['exception']['code']:X}")
        print(f"  异常地址: {metadata['exception']['address']}")
        
    print(f"\n共 {len(metadata['memory_regions'])} 个内存区域")
    return [metadata['header'].timestamp, 
            metadata['system_info'].processor_architecture, 
            metadata['system_info'].major_version, 
            metadata['system_info'].minor_version, 
            metadata['system_info'].build_number, 
            metadata['modules'][:5], 
            metadata['threads'][:3], 
            metadata['exception'], 
            len(metadata['memory_regions'])]


In [6]:
a = meta('proc.dmp')

Minidump分析报告: proc.dmp
创建时间: 1743435527 (Unix时间戳)

系统信息:
  CPU架构: PROCESSOR_ARCHITECTURE_AMD64
  Windows版本: 0.1483145216
  构建号: 131072

加载模块:
  F:\lisarty\Listary\Listary.Service.exe @ 0x210000 (大小: 40960 bytes)
  C:\Windows\System32\ntdll.dll @ 0x7ffe65c90000 (大小: 2191360 bytes)
  C:\Windows\System32\wow64.dll @ 0x7ffe64c00000 (大小: 356352 bytes)
  C:\Windows\System32\wow64base.dll @ 0x7ffe65580000 (大小: 36864 bytes)
  C:\Windows\System32\wow64win.dll @ 0x7ffe64b70000 (大小: 569344 bytes)

线程信息:
  线程ID: 5164, 栈大小: 5816 bytes
  线程ID: 5460, 栈大小: 3656 bytes
  线程ID: 5468, 栈大小: 5016 bytes

共 0 个内存区域


In [4]:
a